In [ ]:
import os
import weaviate
from dotenv import load_dotenv

load_dotenv()

auth_config = weaviate.auth.AuthApiKey(
    api_key="G3YW1TL4j6geVGc0H091LO77ysqTob2z5EbU"
)

In [ ]:
import cohere

# login https://dashboard.cohere.com/api-keys to create COHERE_API_KEY

client = weaviate.Client(
    url="https://3fmi0udzstkt7afklk464g.c0.us-east1.gcp.weaviate.cloud",
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),
    }
)

co = cohere.Client(os.getenv("COHERE_API_KEY"))

In [ ]:
import logging

# Define schema
schema = {
    "classes": [
        {
            "class": "Test",
            "properties": [
                {"name": "title", "dataType": ["text"]},
                {"name": "content", "dataType": ["text"]},
                {"name": "vector", "dataType": ["number[]"]}
            ]
        }
    ]
}

# Check if class 'Test' already exists
existing_classes = client.schema.get()["classes"]
if not any(cls["class"] == "Test" for cls in existing_classes):
    # Create schema in Weaviate
    client.schema.create(schema)
else:
    print("Class 'Test' already exists in Weaviate schema.")

documents = [
    {"title": "Nature 1", "content": "The forest is full of trees."},
    {"title": "Nature 2", "content": "The ocean is vast and blue."},
    {"title": "Nature 3", "content": "Mountains are high and majestic."},
    {"title": "Nature 4", "content": "Rivers flow through the valleys and forest"},
    {"title": "Nature 5", "content": "Deserts are dry and hot."},
    {"title": "Nature 6", "content": "Rainforests are dense and humid."},
    {"title": "Nature 7", "content": "The tundra is cold and barren."},
    {"title": "Nature 8", "content": "Savannas are grassy and open."},
    {"title": "Nature 9", "content": "Lakes are calm and serene."},
    {"title": "Nature 10", "content": "Wetlands are rich in biodiversity."},
    {"title": "Nature 11", "content": "Lot of forest"}
]

# Extract contents for batch embedding
contents = [doc["content"] for doc in documents]

# Embed all documents' contents at once
embeddings = co.embed(texts=contents).embeddings
print(embeddings)

# Add embeddings to documents
for i, doc in enumerate(documents):
    doc["vector"] = embeddings[i]

# Configure batch processing
client.batch.configure(batch_size=10, dynamic=True)

# Add documents to batch
try:
    with client.batch as batch:
        for doc in documents:
            batch.add_data_object(
                data_object=doc,
                class_name="Test"
            )
    logging.info("Documents inserted successfully")
except Exception as e:
    logging.error(f"Error occurred during batch insertion: {e}")

print("Documents inserted successfully")


In [ ]:
query = """
{
  Get {
    Test {
      title
      vector
      _additional {
        id
      }
    }
  }
}
"""

# Execute the query
result = client.query.raw(query)

# Filter documents with non-null vectors and get their titles
titles_with_vectors = [
    doc['title'] for doc in result['data']['Get']['Test'] if doc['vector'] is not None
]

print(f"Titles of documents with vectors: {titles_with_vectors}")

In [ ]:
# Sample query
query = "forest"

In [ ]:
# Function to perform keyword search
def keyword_search(query):
    response = client.query.get("Test", ["title", "content"]).with_where({
        "operator": "Like",
        "path": ["content"],
        "valueText": f"*{query}*"
    }).do()
    return [result["title"] for result in response["data"]["Get"]["Test"]]



# Function to perform vector search
def dense_retrieval(query,
                    client,
                    properties=["title","content"],
                    num_results=5):
    nearVector = {"concepts": [query]}

    # To filter by language
    
    response = (
        client.query
        .get("Test", properties)
        .with_near_vector(nearVector)
        .with_limit(num_results)
        .do()
    )

    return response['data']['Get']['Test']




In [ ]:
# Perform keyword search
keyword_results = keyword_search(query)
print(f"Keyword search results: {keyword_results}")

# Perform vector search
vector_results = dense_retrieval(query,client)
print(f"Vector search results: {vector_results}")


